# Import the dependent libraries

In [491]:
import panel as pn
pn.extension('plotly')
from panel.interact import interact
from panel import widgets
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
from pathlib import Path
from dotenv import load_dotenv #Just in case we need an API key.
import requests
import json
import numpy as np
%matplotlib inline
from datetime import date
from datetime import timedelta
pd.options.display.float_format = '{:.5f}'.format

## Fetch the data from Coinbase API
- TODO
    - Need to show data before dataframe conversion (show our process before creating function)
    - Explain how we got column names
    - 

In [492]:
def fetch_daily_data(symbol, start, end):
    pair_split = symbol.split('/') # Splitting our symbol by the '/' and creating a a list for the new values.
    symbol = pair_split[0] + '-' + pair_split[1] # symbol = BTC-USD #The API request format requires the dash.
    url = f'https://api.pro.coinbase.com/products/{symbol}/candles?start={start}&end={end}&granularity=86400'#notice the symbol insert. There are 86400 seconds in a day.
    response = requests.get(url) #getting response from website
    if response.status_code == 200: # check to make sure the response from server is good
        #if response is good then we create a dataframe by reformatting a json load.
        data = pd.DataFrame(json.loads(response.text), columns=['unix', 'low', 'high', 'open', 'close', 'volume'])
        data['date'] = pd.to_datetime(data['unix'], unit='s') # convert to a readable date
       #######

        # if we failed to get any data, print an error...otherwise write the file
        if data is None:
            print("Did not return any data from Coinbase for this symbol")
        else:
            data.to_csv(f'Coinbase_{pair_split[0] + pair_split[1]}_dailydata_{end}.csv', index=False)
    else:
        print("Did not receieve OK response from Coinbase API")

## Call the fetch function with a function focused on our three main cryptos: BTC/USD, ETH/USD, LTC/USD.

In [493]:
today = date.today()
yesterday = today - timedelta(days = 1)
yesterday = yesterday.strftime("%Y-%m-%d")
#start_date = yesterday - timedelta(days = 300)#wont let me get more then 298 days, or so.
start_date = '2015-01-01'
end_date = '2015-09-30'
#start_date = start_date.strftime("%Y-%m-%d")
#end_date = end_date.strftime("%Y-%m-%d")
cryptolist = ['BTC/USD', 'ETH/USD', 'LTC/USD']

#function to pull crypto data from coinbase api passing in crypto symbol pair, start and end date,
def fetch_main_cryptos(crypto):
    fetch_daily_data(crypto, start_date, end_date)

    
#call the function calling our API loop thrgouh crypto list and pull based on start/end date
for crypto in cryptolist:
    fetch_main_cryptos(crypto)


## Created a path to our newly created CSV files

In [494]:
BTC_path, ETH_path, LTC_path = (Path('../Justin_edits/Coinbase_BTCUSD_dailydata.csv'),
                                Path('../Justin_edits/Coinbase_ETHUSD_dailydata.csv'),
                                Path('../Justin_edits/Coinbase_LTCUSD_dailydata.csv'))
BTC_df, ETH_df, LTC_df = (pd.read_csv(BTC_path, index_col='date', infer_datetime_format=False, parse_dates=True),
                          pd.read_csv(ETH_path, index_col='date', infer_datetime_format=False, parse_dates=True),
                          pd.read_csv(LTC_path, index_col='date', infer_datetime_format=False, parse_dates=True))

## 1

In [495]:
#This function is to create our main datasets. Please edit and comment on how we should approach this.
#user, category, date, polarity, popularity, fav_count, volume, 
def clean_data(df):
    df = df.dropna() # immediately drop any null values
    df = df.drop(columns=['unix']).copy() #create deep copy of df with desired columns
    df['volume_change'] = df['volume'].pct_change() #find daily percent change in volume
    df['percent_volatility'] = round(((df['high'] - df['low']) / df['high']) * 100, 2) #Finding the amount of change between the low and high, then comparing it to the high.
    df['daily_change'] = round(df['close'].pct_change(), 5) # daily pct change
    df['daily_avg_price'] = round((df['open'] + df['close']) / 2, 2)
    df['rolling_volatility'] = round(df['volume'].rolling(30).mean(), 2)
    df['avg_volume_change'] = df['volume_change'].rolling(30).mean()
    df['rolling_volume'] = round(df['volume'].rolling(3).mean(), 2)
    df.drop(df.head(2).index, inplace=True) # drop the unfinished and upcoming day, inclusive of NA data
    df.drop(columns=['low', 'close', 'open'], inplace=True)
    df.sort_index(inplace=True)
    return pd.DataFrame(df)
def clean_new_data(df):
    return df.dropna()
#these are the base data sets so far
BTC_df = clean_data(BTC_df).dropna()
ETH_df = clean_data(ETH_df)
LTC_df = clean_data(LTC_df)

## Current Dataframes

In [496]:
BTC_df.tail()

,high,volume,volume_change,percent_volatility,daily_change,daily_avg_price,rolling_volatility,avg_volume_change,rolling_volume
date,,,,,,,,,
2020-12-17,23776.94000,53602.34952,1.15884,10.62000,-0.01345,22093.49000,34652.70000,0.11668,33003.08000
2020-12-18,23280.00000,24829.23582,0.20661,4.09000,-0.02986,22982.06000,33561.33000,0.08097,20123.44000
2020-12-19,24200.00000,20577.66441,0.37520,5.91000,0.01591,23494.44000,33373.09000,0.07925,20423.24000
2020-12-20,24300.00000,14963.41787,-0.41841,4.94000,0.03287,23663.25000,33240.83000,0.06270,19763.24000
2020-12-21,24118.75000,25728.63856,0.38343,9.14000,-0.04592,23102.96000,33372.20000,0.15677,22579.47000


In [497]:
BTC_df.hvplot(x='date', y='rolling_volume')

:Curve   [date]   (rolling_volume)

## Creating new data frames to compare statistical columns

In [498]:
#joined columns into new dataframe and renamed
BTC_volume = BTC_df['volume']
ETH_volume = ETH_df['volume']
LTC_volume = LTC_df['volume']
volume_df = clean_new_data(pd.concat([BTC_volume, ETH_volume, LTC_volume], axis=1))
volume_df.columns = ['BTC_volume', 'ETH_volume', 'LTC_volume']

#joined columns into new dataframe and renamed
BTC_rolling_volume = BTC_df['rolling_volume']
ETH_rolling_volume = ETH_df['rolling_volume']
LTC_rolling_volume = LTC_df['rolling_volume']
rolling_volume_change_df = clean_new_data(pd.concat([BTC_rolling_volume, ETH_rolling_volume, LTC_rolling_volume], axis=1))
rolling_volume_change_df.columns = ['BTC_rolling_volume', 'ETH_rolling_volume', 'LTC_rolling_volume']


#joined columns into new dataframe and renamed
BTC_volume_change = BTC_df['volume_change']
ETH_volume_change = ETH_df['volume_change']
LTC_volume_change = LTC_df['volume_change']
volume_change_df = clean_new_data(pd.concat([BTC_volume_change, ETH_volume_change, LTC_volume_change], axis=1))
volume_change_df.columns = ['BTC_volume_change', 'ETH_volume_change', 'LTC_volume_change']

#joined columns into new dataframe and renamed
BTC_volatilityr = BTC_df['rolling_volatility']
ETH_volatilityr = ETH_df['rolling_volatility']
LTC_volatilityr = LTC_df['rolling_volatility']
volatilityr_df = clean_new_data(pd.concat([BTC_volatilityr, ETH_volatilityr, LTC_volatilityr], axis=1))
volatilityr_df.columns = ['BTC_rolling_volatility', 'ETH_rolling_volatility', 'LTC_rolling_volatility']

#joined columns into new dataframe and renamed
BTC_volatility = BTC_df['percent_volatility']
ETH_volatility = ETH_df['percent_volatility']
LTC_volatility = LTC_df['percent_volatility']
volatility_df = clean_new_data(pd.concat([BTC_volatility, ETH_volatility, LTC_volatility], axis=1))
volatility_df.columns = ['BTC_volatility', 'ETH_volatility', 'LTC_volatility']

#joined columns into new dataframe and renamed
BTC_high = BTC_df['high']
ETH_high = ETH_df['high']
LTC_high = LTC_df['high']
close_df = clean_new_data(pd.concat([BTC_high, ETH_high, LTC_high], axis=1))
close_df.columns = ['BTC_high', 'ETH_high', 'LTC_high']
ETH_LTC_close_df = clean_new_data(close_df.drop(columns='BTC_high'))

#joined columns into new dataframe and renamed
BTC_daily_change = BTC_df['daily_change']
ETH_daily_change = ETH_df['daily_change']
LTC_daily_change = LTC_df['daily_change']
daily_change_df = clean_new_data(pd.concat([BTC_daily_change, ETH_daily_change, LTC_daily_change], axis=1))
daily_change_df.columns = ['BTC_daily_change', 'ETH_daily_change', 'LTC_daily_change']


## Show new Dataframes

## Create functions for hvplots so they can be combined into a dashboard

In [499]:
#shows us the pct change of daily volume
def volume_change():
    return volume_change_df.hvplot(ylim=(-1, 3))
#shows us trend line of high and low volatility
def volatility():
    return volatility_df.hvplot()
#daily stock prices
def close():
    return high_df.hvplot()
#daily stock prices (ETH and LTC)
def zoom_in_close():
    return ETH_LTC_close_df.hvplot()
#daily pct change
def pct_change_plot():
    return daily_change_df.hvplot()

In [500]:
#volume_change_df.hvplot(ylim=(-1, 10))

In [501]:
#volatility_df.hvplot()

In [502]:
#close_df.hvplot()

In [503]:
#ETH_LTC_close_df.hvplot()

In [504]:
#daily_change_df.hvplot()

# Created hypothetical tabs for dashboard

In [505]:
#tab_1 = pn.Column(
    #volume_change(),
    #volatility(),
    #close(),
    #zoom_in_close(),
    #pct_change_plot()
#)
#dashboard = pn.Tabs(
 #   ('tab_1', tab_1)
#)

In [506]:
#dashboard.servable()

# Pull twitter data, then clean and create merged dataset

In [507]:
twitter_path = Path('../data/raw_data/raw_tweets_01_filter_polarity.csv')
twitter_df = pd.read_csv(twitter_path, infer_datetime_format=True, parse_dates=True)
datetime = twitter_df['time'].str.split(" ", n=1, expand = True)
twitter_df['date'] = datetime[0]
twitter_df.drop(columns='time')
twitter_df['time'] = datetime[1]
twitter_df.drop_duplicates(subset='text', inplace=True)
twitter_df.set_index('date', inplace=True)
#TODO: round all columns
df = pd.merge(twitter_df, volume_change_df, how='inner', left_index=True, right_index=True)
df2 = pd.merge(df, volatility_df, how='inner', left_index=True, right_index=True)
df3 = pd.merge(df2, close_df, how='inner', left_index=True, right_index=True)
df4 = pd.merge(df3, volatilityr_df, how='inner', left_index=True, right_index=True)
df5 = pd.merge(df4, rolling_volume_change_df, how='inner', left_index=True, right_index=True)
df6 = pd.merge(df5, volume_df, how='inner', left_index=True, right_index=True)
twitter_analysis_df = pd.merge(df6, daily_change_df, how='inner', left_index=True, right_index=True)
twitter_analysis_df['Popularity Rating'] = round(twitter_analysis_df['Popularity Rating'], 2)
twitter_analysis_df.sort_index(inplace=True)
twitter_analysis_df['category'].fillna('null', inplace=True)

#tweet_analysis_df['2020-03-30':'2021-19-01']
#df1.merge(df2, on='ID', how='left')
def export_df(df):
    return df.to_csv('../data/raw_data/raw_crypto_data.csv')
export_df(twitter_analysis_df)

# Here is some example data. We can add any tables and mess with the data from here. Anything we want to add on we should do through our crypto data frames and restart kernel

In [508]:
twitter_users = twitter_analysis_df['twitter_user'].unique()
crypto_category = twitter_analysis_df['category'].unique()
crypto_category
bitcoin_tweets_df = twitter_analysis_df[twitter_analysis_df['category'].str.contains('bit')]
ethereum_tweets_df = twitter_analysis_df[twitter_analysis_df['category'].str.contains('eth')]
twitter_users


array(['VitalikButerin', 'officialmcafee', 'joerogan', 'jack',
       'SatoshiLite', 'elonmusk'], dtype=object)

In [509]:
avg_df = twitter_analysis_df[twitter_analysis_df['category'] == 'bitcoin']
avg_df.head()

,twitter_user,category,time,sentiment,text,tweet_id,tweet_source,quote_count,reply_count,retweet_count,...,LTC_rolling_volatility,BTC_rolling_volume,ETH_rolling_volume,LTC_rolling_volume,BTC_volume,ETH_volume,LTC_volume,BTC_daily_change,ETH_daily_change,LTC_daily_change
date,,,,,,,,,,,,,,,,,,,,,
2016-08-22,VitalikButerin,bitcoin,06:03:10+00:00,nan,RT @el33th4xor: Which open source project gene...,767602996293218304,Twitter Web Client,0,0,0,...,1431.61000,4480.91000,30998.12000,3601.18000,4493.44111,36422.56390,0.62469,0.00482,0.00909,0.01013
2016-08-24,VitalikButerin,bitcoin,23:55:44+00:00,nan,@zooko @BitcoinErrorLog If you want my respons...,768597693601951744,Twitter Web Client,1,1,2,...,1403.84000,3773.44000,26758.38000,4501.89000,3857.02319,24028.11263,5183.13519,0.00502,-0.02910,-0.00525
2016-08-26,officialmcafee,bitcoin,01:23:09+00:00,nan,Cooling system for our Bitcoin mining operatio...,768982079954485248,Twitter for Android,7,10,38,...,1098.22000,3000.38000,21910.61000,2127.56000,3354.18317,27315.07631,3336.89310,0.01480,0.00894,0.00794
2016-09-02,officialmcafee,bitcoin,22:38:08+00:00,nan,Odd things like this are materializing at our ...,771839655197155328,Twitter for Android,9,38,33,...,933.57000,4709.40000,32465.87000,1868.29000,4827.43460,40624.95260,2725.27021,-0.04110,0.02273,-0.01031
2016-09-07,joerogan,bitcoin,19:43:13+00:00,nan,About to go live with Bitcoin Buddha Andreas A...,773607576244252672,Instagram,4,19,44,...,1567.25000,4802.95000,28996.55000,326.56000,4680.95772,33512.36863,502.70085,-0.02032,0.01668,-0.00998


In [510]:
elon = twitter_users[5]
elon_tweets_df = twitter_analysis_df[twitter_analysis_df['twitter_user'] == elon]
elon_tweets_rated = elon_tweets_df[elon_tweets_df['Popularity Rating'] != 0]
elon_tweets_rated = elon_tweets_df[elon_tweets_df['category'] == 'bitcoin']
elon_tweets_rated.drop(columns=['sentiment', 'time', 'tweet_id', 'tweet_source', 'quote_count', 'reply_count', 'retweet_count'], inplace=True)


C:\Users\miche\anaconda3\lib\site-packages\pandas\core\frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [511]:
elon_tweets_rated.columns

Index(['twitter_user', 'category', 'text', 'fav_count', 'Polarity Rating',
       'Popularity Rating', 'BTC_volume_change', 'ETH_volume_change',
       'LTC_volume_change', 'BTC_volatility', 'ETH_volatility',
       'LTC_volatility', 'BTC_high', 'ETH_high', 'LTC_high',
       'BTC_rolling_volatility', 'ETH_rolling_volatility',
       'LTC_rolling_volatility', 'BTC_rolling_volume', 'ETH_rolling_volume',
       'LTC_rolling_volume', 'BTC_volume', 'ETH_volume', 'LTC_volume',
       'BTC_daily_change', 'ETH_daily_change', 'LTC_daily_change'],
      dtype='object')

In [516]:
elon_tweets_rated['Popularity Rating'].apply(lambda x: '%.f' % x)

date
2017-11-28      -688
2018-10-22         0
2019-02-21       622
2020-01-10     60602
2020-05-01      3435
2020-05-01         0
2020-05-15      1684
2020-11-16         0
2020-12-20         0
2020-12-20    121569
Name: Popularity Rating, dtype: object

In [518]:
elon_tweets_rated.hvplot.scatter(x='date', y='Popularity Rating', width=2000, height=500, yformatter='%.0f') * elon_tweets_rated.hvplot.line(x='date', y='BTC_rolling_volume') * elon_tweets_rated.hvplot.area(x='date', y='BTC_volume', stacked=False)

:Overlay
   .Scatter.I :Scatter   [date]   (Popularity Rating)
   .Curve.I   :Curve   [date]   (BTC_rolling_volume)
   .Area.I    :Area   [date]   (BTC_volume)

In [412]:
elon_tweets_rated.hvplot.line(x='date', y='BTC_rolling_volume')

:Curve   [date]   (BTC_rolling_volume)

In [326]:
elon_tweets_df.hvplot.bar(x='date', y=['BTC_volatility', 'ETH_volatility', 'LTC_volatility'], rot=90)

:Bars   [date,Variable]   (value)

# Elon has limited tweets, so here is 

In [247]:
elon_tweets_df.hvplot.bar(x='date', y=['BTC_volume_change', 'ETH_volume_change', 'LTC_volume_change'], groupby='date.year', rot=90) + elon_tweets_df.hvplot.bar(x='date', y=['Polarity Rating'], rot=90)

:Layout
   .DynamicMap.I :DynamicMap   [date.year]
      :Bars   [date,Variable]   (value)
   .Bars.I       :Bars   [date,Variable]   (value)

In [248]:
mcafee = twitter_users[1]
mcafee_tweets_df = twitter_analysis_df[twitter_analysis_df['twitter_user'] == mcafee]
mcafee_tweets_positive = mcafee_tweets_df[mcafee_tweets_df['Polarity Rating'] >= .75]
mcafee_tweets_negative = mcafee_tweets_df[mcafee_tweets_df['Polarity Rating'] <= -.75]
mcafee_polarity_df = pd.concat([mcafee_tweets_positive, mcafee_tweets_negative])
mcafee_polarity_df['BTC_volume'] = BTC_df['volume']
mcafee_polarity_df.columns

Index(['twitter_user', 'category', 'time', 'sentiment', 'text', 'tweet_id',
       'tweet_source', 'quote_count', 'reply_count', 'retweet_count',
       'fav_count', 'Polarity Rating', 'Popularity Rating',
       'BTC_volume_change', 'ETH_volume_change', 'LTC_volume_change',
       'BTC_volatility', 'ETH_volatility', 'LTC_volatility', 'BTC_high',
       'ETH_high', 'LTC_high', 'BTC_rolling_volatility',
       'ETH_rolling_volatility', 'LTC_rolling_volatility', 'BTC_daily_change',
       'ETH_daily_change', 'LTC_daily_change', 'BTC_volume'],
      dtype='object')

In [249]:
mcafee_polarity_df.hvplot.bar(x='date', y=['BTC_volume_change', 'ETH_volume_change', 'LTC_volume_change'], groupby='date.year', rot=90) + mcafee_polarity_df.hvplot.bar(x='date', y='Polarity Rating', groupby='date.year', rot=90)

:Layout
   .DynamicMap.I  :DynamicMap   [date.year]
      :Bars   [date,Variable]   (value)
   .DynamicMap.II :DynamicMap   [date.year]
      :Bars   [date]   (Polarity Rating)

In [250]:
twitter_analysis_df.hvplot(x='date', y=['BTC_rolling_volatility', 'ETH_rolling_volatility', 'LTC_rolling_volatility'], rot=90)

:NdOverlay   [Variable]
   :Curve   [date]   (value)

In [226]:
mcafee_polarity_df.hvplot.bar(x='date', y=['BTC_volatility', 'ETH_volatility', 'LTC_volatility'], rot=90, groupby='date.year', height=500, width=2000)# + mcafee_polarity_df.hvplot.scatter(x='date', y='retweet_count', groupby='date.year')

:DynamicMap   [date.year]
   :Bars   [date,Variable]   (value)

In [40]:
mcafee_polarity_df.hvplot.bar(x='date', y=['Polarity Rating'], rot=90, groupby='date.year', height=500, width=2000)

:DynamicMap   [date.year]
   :Bars   [date,Variable]   (value)